# Movie Rating Hypothesis testing

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


In this assignment we are going prove of disprove a hypothesis

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from IPython.display import HTML, display

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 25)

In [3]:
%matplotlib inline

##### Read and cleanup the movie data

In [4]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames, encoding='latin-1')
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames, encoding='latin-1')
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames, encoding='latin-1')

    return users, ratings, movies

users, ratings, movies = get_movie_data()

tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

print(movies.head())

C:\Users\varsh\AppData\Local\Temp\ipykernel_16516\2599110875.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_table(os.path.join('../data','users.dat'),
C:\Users\varsh\AppData\Local\Temp\ipykernel_16516\2599110875.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table(os.path.join('../data', 'ratings.dat'),


   movie_id                               title                        genres  \
0         1                    Toy Story (1995)   Animation|Children's|Comedy   
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2         3             Grumpier Old Men (1995)                Comedy|Romance   
3         4            Waiting to Exhale (1995)                  Comedy|Drama   
4         5  Father of the Bride Part II (1995)                        Comedy   

   year                  short_title  
0  1995                    Toy Story  
1  1995                      Jumanji  
2  1995             Grumpier Old Men  
3  1995            Waiting to Exhale  
4  1995  Father of the Bride Part II  


C:\Users\varsh\AppData\Local\Temp\ipykernel_16516\2599110875.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table(os.path.join('../data', 'movies.dat'),


### 1. Prove or disprove the following Null Hypothesis:

#### H0: Different age segments of users rate the different genres of movies similarly

- Age segments are (0-20), (21-30), (31-50), (50 and above),
- Genres - Drama, Comedy, Action, Romance, Adventure
- Rating 4 or above is a positive, else negative (used for binary votes)

#### Solution


#### Im joining the movies and ratings dataframes and setting the index a movie_id to create dummies to seperate the genres column
> ***movies_ratings*** dataframe

In [5]:
movies_ratings = (ratings
                  .set_index("movie_id")
                  .join(movies.set_index("movie_id"),
                        how="left")
                 )
movies_ratings.shape

(1000209, 7)

#### Creating seperate columns for genres by using get_dummies as they are all in one column seperated by pipe delimiter
> ***genres_dummies*** dataframe

In [6]:
genres_dummies = movies_ratings['genres'].str.get_dummies()
genres_dummies.shape

(1000209, 18)

#### Concatenate these dummies to the original movies_ratings data frame
> **full_movies_ratings** dataframe
- Also dropping genres other than Drama, Comedy, Action, Romance, Adventure

In [7]:
full_movies_ratings = (pd.concat([movies_ratings, genres_dummies], axis=1)
                       .drop(["timestamp","Animation","Children's","Crime","Documentary","Fantasy","Film-Noir","Horror","Musical","Mystery","Sci-Fi","Thriller","War","Western"], axis=1)
                )

full_movies_ratings.head()

          user_id  rating             title                       genres  \
movie_id                                                                   
1               1       5  Toy Story (1995)  Animation|Children's|Comedy   
1               6       4  Toy Story (1995)  Animation|Children's|Comedy   
1               8       4  Toy Story (1995)  Animation|Children's|Comedy   
1               9       5  Toy Story (1995)  Animation|Children's|Comedy   
1              10       5  Toy Story (1995)  Animation|Children's|Comedy   

          year short_title  Action  Adventure  Comedy  Drama  Romance  
movie_id                                                               
1         1995   Toy Story       0          0       1      0        0  
1         1995   Toy Story       0          0       1      0        0  
1         1995   Toy Story       0          0       1      0        0  
1         1995   Toy Story       0          0       1      0        0  
1         1995   Toy Story       0 

In [8]:
full_movies_ratings.reset_index(inplace=True)
full_movies_ratings.shape

(1000209, 12)

#### Merging users datafram with full_movies_ratings on user_id as common column
> ***full_users_ratings_movies*** dataframe

In [9]:
full_users_ratings_movies = pd.merge(users, full_movies_ratings, on=['user_id'], how='inner').drop(["occupation", "zip","gender"],axis=1)
full_users_ratings_movies.head()

   user_id  age  movie_id  rating                                      title  \
0        1    1         1       5                           Toy Story (1995)   
1        1    1        48       5                          Pocahontas (1995)   
2        1    1       150       5                           Apollo 13 (1995)   
3        1    1       260       4  Star Wars: Episode IV - A New Hope (1977)   
4        1    1       527       5                    Schindler's List (1993)   

                                 genres  year  \
0           Animation|Children's|Comedy  1995   
1  Animation|Children's|Musical|Romance  1995   
2                                 Drama  1995   
3       Action|Adventure|Fantasy|Sci-Fi  1977   
4                             Drama|War  1993   

                          short_title  Action  Adventure  Comedy  Drama  \
0                           Toy Story       0          0       1      0   
1                          Pocahontas       0          0       0      0   

In [10]:
full_users_ratings_movies.shape

(1000209, 13)

# 1. Prove or disprove the following Null Hypothesis:

## H0: Different age segments of users rate the different genres of movies similarly

### - Rating 4 or above is a positive, else negative (used for binary votes)

#### Converting the ratings into binary vote
> The ratings given to the movies from 1 to 5 is concerted to binary rating that is ratings 4 or above then the rating is positive or one 
else if the rating is below 4 then the rating is negative or 0 .

In [11]:
def my_function(x):
    if x >= 4:
        return 1
    if x < 4:
        return 0
binary_vote=pd.DataFrame(full_users_ratings_movies["rating"].map(my_function))
binary_vote=binary_vote.rename({'rating': 'Binary_rating'}, axis='columns')

In [12]:
binary_vote

         Binary_rating
0                    1
1                    1
2                    1
3                    1
4                    1
...                ...
1000204              1
1000205              1
1000206              1
1000207              1
1000208              1

[1000209 rows x 1 columns]

> Concatenating binary_vote dataframe into full_users_ratings_movies dataframe 

In [13]:
users_ratings_movies = (pd.concat([full_users_ratings_movies,binary_vote], axis=1))
users_ratings_movies.head()

   user_id  age  movie_id  rating                                      title  \
0        1    1         1       5                           Toy Story (1995)   
1        1    1        48       5                          Pocahontas (1995)   
2        1    1       150       5                           Apollo 13 (1995)   
3        1    1       260       4  Star Wars: Episode IV - A New Hope (1977)   
4        1    1       527       5                    Schindler's List (1993)   

                                 genres  year  \
0           Animation|Children's|Comedy  1995   
1  Animation|Children's|Musical|Romance  1995   
2                                 Drama  1995   
3       Action|Adventure|Fantasy|Sci-Fi  1977   
4                             Drama|War  1993   

                          short_title  Action  Adventure  Comedy  Drama  \
0                           Toy Story       0          0       1      0   
1                          Pocahontas       0          0       0      0   

### - Age segments (0-20), (21-30), (31-50), (50 and above)
#### Codification of Age into segments
> dividing the Age column into 4 different segments of ranges (0-20),(21-30),(31-50),(50-above) and storing it as a seperate Age_bin column

In [14]:
bins = [0,21,31,50, np.inf]
labels = ["0-20","21-30","31-50","50-above"]
users_ratings_movies['Age_bin'] = pd.cut(users_ratings_movies['age'], bins=bins, labels=labels)

Age_group = users_ratings_movies.groupby(['Age_bin'])['Binary_rating'].agg(['count'])
Age_group

           count
Age_bin         
0-20      210747
21-30     395556
31-50     355126
50-above   38780

### - Genres - Drama, Comedy, Action, Romance, Adventure

#### Im considering all the movies form the respective movie genres
> Adding a condition to seperate out all the Action, Adventure, Comedy, Drama, Romance movies and also getting their Binary votes respectively.

##### Action
> Getting all action genre metioned movies and storing their binary votes into separate column

In [15]:
cols_Action= ["movie_id","age","Age_bin","Action","Binary_rating"]
condition_Action = users_ratings_movies["Action"] ==1

Action = pd.DataFrame(users_ratings_movies
         [cols_Action]
         [condition_Action].rename({'Binary_rating':'Action_Binary'}, axis='columns'))
Action.head()

    movie_id  age Age_bin  Action  Action_Binary
3        260    1    0-20       1              1
23      1197    1    0-20       1              0
27      1287    1    0-20       1              1
36      2028    1    0-20       1              1
43      2692    1    0-20       1              1

Sum of all binary votes for action genre

In [16]:
Action['Action_Binary'].sum()

138766

groupby Age_bin and count the binary vote for the action genre with respect to the respective age segments

In [17]:
Action_group = pd.DataFrame(Action.groupby(['Age_bin'])['Action_Binary'].agg(['count']).rename({'count':'ACTION'}, axis='columns'))
Action_group

          ACTION
Age_bin         
0-20       56764
21-30     105678
31-50      86872
50-above    8143

##### Adventure

> Getting all adventure genre metioned movies and storing their binary votes into separate column

In [18]:
cols_Adventure= ["movie_id","age","Age_bin","Adventure","Binary_rating"]
condition_Adventure = users_ratings_movies["Adventure"] ==1

Adventure = pd.DataFrame(users_ratings_movies
         [cols_Adventure]
         [condition_Adventure].rename({'Binary_rating':'Adventure_Binary'}, axis='columns'))
Adventure.head()

    movie_id  age Age_bin  Adventure  Adventure_Binary
3        260    1    0-20          1                 1
15       919    1    0-20          1                 1
23      1197    1    0-20          1                 0
27      1287    1    0-20          1                 1
29      1566    1    0-20          1                 1

Sum of all binary votes for comedy genre

In [19]:
Adventure['Adventure_Binary'].sum()

70951

groupby Age_bin and count the binary vote for the adventure genre with respect to the respective age segments

In [20]:
Adventure_group = pd.DataFrame(Adventure.groupby(['Age_bin'])['Adventure_Binary'].agg(['count']).rename({'count':'ADVENTURE'}, axis='columns'))
Adventure_group

          ADVENTURE
Age_bin            
0-20          30322
21-30         52633
31-50         46510
50-above       4488

##### Comedy

> Getting all Comedy genre metioned movies and storing their binary votes into separate column

In [21]:
cols_Comedy= ["movie_id","age","Age_bin","Comedy","Binary_rating"]
condition_Comedy = users_ratings_movies["Comedy"] ==1

Comedy = pd.DataFrame(users_ratings_movies
         [cols_Comedy]
         [condition_Comedy].rename({'Binary_rating':'Comedy_Binary'}, axis='columns'))
Comedy.head()

    movie_id  age Age_bin  Comedy  Comedy_Binary
0          1    1    0-20       1              1
6        588    1    0-20       1              1
12       745    1    0-20       1              0
18      1028    1    0-20       1              1
23      1197    1    0-20       1              0

Sum of all binary votes for comedy genre

In [22]:
Comedy['Comedy_Binary'].sum()

196945

groupby Age_bin and count the binary vote for the comedy genre with respect to the respective age segments

In [23]:
Comedy_group = pd.DataFrame(Comedy.groupby(['Age_bin'])['Comedy_Binary'].agg(['count']).rename({'count':'COMEDY'}, axis='columns'))
Comedy_group

          COMEDY
Age_bin         
0-20       81142
21-30     143210
31-50     120267
50-above   11961

##### Drama

> Getting all Drama genre metioned movies and storing their binary votes into separate column

In [24]:
cols_Drama= ["movie_id","age","Age_bin","Drama","Binary_rating"]
condition_Drama = users_ratings_movies["Drama"] ==1

Drama = pd.DataFrame(users_ratings_movies
         [cols_Drama]
         [condition_Drama].rename({'Binary_rating':'Drama_Binary'}, axis='columns'))
Drama.head()

    movie_id  age Age_bin  Drama  Drama_Binary
2        150    1    0-20      1             1
4        527    1    0-20      1             1
5        531    1    0-20      1             1
9        608    1    0-20      1             1
15       919    1    0-20      1             1

Sum of all binary votes for drama genre

In [25]:
Drama['Drama_Binary'].sum()

228440

groupby Age_bin and count the binary vote for the drama genre with respect to the respective age segments

In [26]:
Drama_group = pd.DataFrame(Drama.groupby(['Age_bin'])['Drama_Binary'].agg(['count']).rename({'count':'DRAMA'}, axis='columns'))
Drama_group

           DRAMA
Age_bin         
0-20       65587
21-30     138695
31-50     132978
50-above   17269

##### Romance

> Getting all romance genre metioned movies and storing their binary votes into separate column

In [27]:
cols_Romance= ["movie_id","age","Age_bin","Romance","Binary_rating"]
condition_Romance = users_ratings_movies["Romance"] ==1

Romance = pd.DataFrame(users_ratings_movies
         [cols_Romance]
         [condition_Romance].rename({'Binary_rating':'Romance_Binary'}, axis='columns'))
Romance.head()

    movie_id  age Age_bin  Romance  Romance_Binary
1         48    1    0-20        1               1
14       914    1    0-20        1               0
23      1197    1    0-20        1               0
30      1721    1    0-20        1               1
39      2340    1    0-20        1               0

Sum of all binary votes for romance genre

In [28]:
Romance['Romance_Binary'].sum()

85499

groupby Age_bin and count the binary vote for the romance genre with respect to the respective age segments

In [29]:
Romance_group = pd.DataFrame(Romance.groupby(['Age_bin'])['Romance_Binary'].agg(['count']).rename({'count':'ROMANCE'}, axis='columns'))
Romance_group

          ROMANCE
Age_bin          
0-20        29255
21-30       58003
31-50       53986
50-above     6279

#### Merge all 5 genres counts and the age_bins

Merge Action_group and Adventure_group to final_df 

In [30]:
final_df = pd.merge(Action_group,Adventure_group, on=['Age_bin'], how='inner')
final_df.head()

          ACTION  ADVENTURE
Age_bin                    
0-20       56764      30322
21-30     105678      52633
31-50      86872      46510
50-above    8143       4488

Merge final_df and Comedy_group to final_df

In [31]:
final_df = pd.merge(final_df,Comedy_group, on=['Age_bin'], how='inner')
final_df.head()

          ACTION  ADVENTURE  COMEDY
Age_bin                            
0-20       56764      30322   81142
21-30     105678      52633  143210
31-50      86872      46510  120267
50-above    8143       4488   11961

Merge final_df and Drama_group to final_df

In [32]:
final_df = pd.merge(final_df,Drama_group, on=['Age_bin'], how='inner')
final_df.head()

          ACTION  ADVENTURE  COMEDY   DRAMA
Age_bin                                    
0-20       56764      30322   81142   65587
21-30     105678      52633  143210  138695
31-50      86872      46510  120267  132978
50-above    8143       4488   11961   17269

Merge final_df and Romance_group to final_df

In [33]:
final_df = pd.merge(final_df,Romance_group, on=['Age_bin'], how='inner')
final_df.head()

          ACTION  ADVENTURE  COMEDY   DRAMA  ROMANCE
Age_bin                                             
0-20       56764      30322   81142   65587    29255
21-30     105678      52633  143210  138695    58003
31-50      86872      46510  120267  132978    53986
50-above    8143       4488   11961   17269     6279

> As knowing the value of Age_bin tells us nothing about the value of genres.
So , they are indipendent variables

####  Get Both Column Total and Row Total

Appending column total that is sum of all columns and row total that is sum of all rows

In [34]:
final_df.loc['Column_Total']= final_df.sum(numeric_only=True, axis=0)
final_df.loc[:,'Row_Total'] = final_df.sum(numeric_only=True, axis=1)
final_df

              ACTION  ADVENTURE  COMEDY   DRAMA  ROMANCE  Row_Total
Age_bin                                                            
0-20           56764      30322   81142   65587    29255     263070
21-30         105678      52633  143210  138695    58003     498219
31-50          86872      46510  120267  132978    53986     440613
50-above        8143       4488   11961   17269     6279      48140
Column_Total  257457     133953  356580  354529   147523    1250042

 #### Observed
 Get table without Row_Total and Column_Total for later use

In [35]:
observed = final_df.iloc[0:4,0:5]  
observed

          ACTION  ADVENTURE  COMEDY   DRAMA  ROMANCE
Age_bin                                             
0-20       56764      30322   81142   65587    29255
21-30     105678      52633  143210  138695    58003
31-50      86872      46510  120267  132978    53986
50-above    8143       4488   11961   17269     6279

#### Expected
To get the expected count for a cell, we multiply the Row_Total for that cell by the Column_Total for that cell and then divide by the total number of observations.


In [36]:
expected =  np.outer(final_df["Row_Total"][0:4],
                     final_df.loc["Column_Total"][0:5]) / 1250042

expected = pd.DataFrame(expected)

expected.columns = ["ACTION","ADVENTURE","COMEDY","DRAMA","ROMANCE"]
expected.index = ["0-20","21-30","31-50","50-above"]

expected

                 ACTION     ADVENTURE         COMEDY          DRAMA  \
0-20       54181.549892  28190.265375   75041.879073   74610.248320   
21-30     102612.527485  53388.549910  142119.169612  141301.719343   
31-50      90748.071778  47215.560108  125686.803755  124963.870236   
50-above    9914.850845   5158.624606   13732.147560   13653.162102   

               ROMANCE  
0-20      31046.057340  
21-30     58797.033649  
31-50     51998.694123  
50-above   5681.214887  

#### chi-squared 
chi-squared formula is the summation of observed minus expected values whole square ,divived by expected value

In [43]:
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()

print(chi_squared_stat)

4796.687892530289


- degrees of freedom for a test of independence equals the product of the number of categories in each variable minus 1.
 4x5 table so df = 3x4 = 12.
- Find the critical value for 95% confidence
- Find the p-value

In [40]:
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 12)   # *

print("Critical value")
print(crit)

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=12)
print("P value")
print(p_value)

Critical value
21.02606981748307
P value
0.0


test of independence automatically given a frequency table of observed counts

In [41]:
stats.chi2_contingency(observed= observed)

(4796.687892530289,
 0.0,
 12,
 array([[ 54181.54989192,  28190.26537508,  75041.87907286,
          74610.24831966,  31046.05734047],
        [102612.52748548,  53388.54991032, 142119.1696119 ,
         141301.71934303,  58797.03364927],
        [ 90748.07177759,  47215.56010838, 125686.80375539,
         124963.87023556,  51998.69412308],
        [  9914.85084501,   5158.62460621,  13732.14755984,
          13653.16210175,   5681.21488718]]))

The output shows the chi-square statistic, the p-value and the degrees of freedom followed by the expected count.

Given the null p-value,The test result dose detect a significant relationship between the variables

Since our chi-squared statistics exceeds the critical value , we would **Reject the null hypothesis that is "Different age segments of users rate the different genres of movies differently"**